In [1]:
import mne
import os
import sys
sys.path.insert(0,'../')
from libs.ssl_dataloader import *
from libs.ssl_model import *
from sklearn.model_selection import train_test_split

In [4]:
def get_subjects(basedir):
    files = np.array([i for i in os.listdir(basedir) if i.split('.')[-1] == 'set'])
    subjects = np.array([i.split('_')[0] for i in os.listdir(basedir) if i.split('.')[-1] == 'set'])
    return subjects, files

subjects, files = get_subjects("/expanse/projects/nemar/child-mind-rest")
subj_train, subj_test, files_train, files_test = train_test_split(subjects, files, test_size=0.1, random_state=1)

subj_train, subj_val, files_train, files_val = train_test_split(subj_train, files_train, test_size=0.22, random_state=1) # 0.22 x 0.9 = 0.2



In [2]:
SFREQ = 128
dataset = ChildmindSSLDataset(
        data_dir = "/expanse/projects/nemar/child-mind-rest",
        # subjects = subj_train.tolist(),
        x_params = {
            "feature": "RelativePositioning",
            "cache_dir": '/expanse/projects/nemar/dtyoung/eeg-ssl/data/childmind-rest-cache',
            "win": SFREQ*2,
            "stride": SFREQ,
            "tau_pos": int(SFREQ*6),
            "tau_neg": int(SFREQ*6),
            "isResume": False
        }
    )

  0%|          | 0/2258 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 64 concurrent workers.
  6%|▌         | 128/2258 [00:00<00:02, 1059.61it/s]/expanse/projects/nemar/dtyoung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:70: RuntimeWarning: Estimated head radius (0.9 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  EEG = mne.io.read_raw_eeglab(os.path.join(raw_file), preload=True)
/expanse/projects/nemar/dtyoung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:70: RuntimeWarning: Estimated head radius (0.9 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  EEG = mne.io.read_raw_eeglab(os.path.join(raw_file), preload=True)
/expanse/projects/nemar/dtyoung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:70: RuntimeWarning: Estimated head radius (0.9 cm)

In [ ]:
print('Length of dataset', len(dataset))
print('X dim', dataset[0][0].shape)
print('Y dim', dataset[0][1].shape)

In [3]:
model_utils = SSLModelUtils(model_params={'model': 'VGGSSL', 'task': 'RP'}, train_params={'num_epochs': 5})
model_utils.train(dataset)

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 147456 bytes. Error code 12 (Cannot allocate memory)